# Window Query

In [3]:
// Package to read data from FITS file
%AddDeps com.github.astrolabsoftware spark-fits_2.11 0.6.0

// Smile provides visualisation tools
%AddDeps com.github.haifengl smile-plot 1.5.1
%AddDeps com.github.haifengl smile-math 1.5.1
%AddDeps com.github.haifengl smile-core 1.5.1
%AddDeps com.github.haifengl smile-scala_2.11 1.5.1

// Contains extensions to the Swing GUI toolkit
%AddDeps org.swinglabs swingx 1.6.1

// Add the spark3d JAR. To generate it, run `sbt ++2.11.8` package at the root of the package
%AddJar file:/Users/julien/Documents/workspace/myrepos/spark3D/target/scala-2.11/spark3d_2.11-0.1.5.jar

// Add healpix JAR
%AddJar file:/Users/julien/Documents/workspace/myrepos/spark3D/lib/jhealpix.jar

Marking com.github.astrolabsoftware:spark-fits_2.11:0.6.0 for download
Preparing to fetch from:
-> file:/var/folders/my/lfvl285927q2hzk545f39sy40000gn/T/toree_add_deps2488765369524960292/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/my/lfvl285927q2hzk545f39sy40000gn/T/toree_add_deps2488765369524960292/https/repo1.maven.org/maven2/com/github/astrolabsoftware/spark-fits_2.11/0.6.0/spark-fits_2.11-0.6.0.jar
Marking com.github.haifengl:smile-plot:1.5.1 for download
Preparing to fetch from:
-> file:/var/folders/my/lfvl285927q2hzk545f39sy40000gn/T/toree_add_deps2488765369524960292/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/my/lfvl285927q2hzk545f39sy40000gn/T/toree_add_deps2488765369524960292/https/repo1.maven.org/maven2/com/github/haifengl/smile-plot/1.5.1/smile-plot-1.5.1.jar
Marking com.github.haifengl:smile-math:1.5.1 for download
Preparing to fetch from:
-> file:/var/folders/my/lfvl285927q2hzk545f39sy40000gn/T/toree_add_deps2488765369524960292/
-> htt

# From raw data RDD to Point3D RDD

Load data from the test files provided in the spark3d repo.
Our raw data contains points with 3D coordinates (spherical: r, theta, phi) sampled from uniform distributions. Let's transform it into a Point3D RDD

In [5]:
import com.astrolabsoftware.spark3d.spatial3DRDD._
import org.apache.spark.sql.SparkSession
import com.astrolabsoftware.spark3d.spatialOperator.RangeQuery
import com.astrolabsoftware.spark3d.geometryObjects.ShellEnvelope
import com.astrolabsoftware.spark3d.geometryObjects.BoxEnvelope
import com.astrolabsoftware.spark3d.geometryObjects.Point3D
import com.astrolabsoftware.spark3d.utils.Utils.cartesianToSpherical
import com.astrolabsoftware.spark3d.utils.Utils.sphericalToCartesian

val spark = SparkSession.builder().appName("query").getOrCreate()

val fnA = "../../src/test/resources/astro_obs.fits"
val options = Map("hdu" -> "1")

val columns = "Z_COSMO,RA,DEC"
val spherical = true

// Load the data
val pointRDDA = new Point3DRDD(spark, fnA, columns, spherical, "fits", options)

// Perform the query
val p = new Point3D(0.9, 0.0, 0.0, true)
val sphere = new ShellEnvelope(p, 0.1)
val box = BoxEnvelope(-0.5, 0.0, -0.5, 0.0, -0.5, 0.0)
val sphereMatches = RangeQuery.windowQuery(pointRDDA.rawRDD, sphere)
val boxMatches = RangeQuery.windowQuery(pointRDDA.rawRDD.map(sphericalToCartesian(_)), box).map(cartesianToSpherical(_))

# Display the results

In [6]:
import smile.plot._
import java.awt.Color
import java.awt.{GridLayout, Dimension}

import javax.swing.JFrame
import javax.swing.JPanel

import com.astrolabsoftware.spark3d.utils.Utils.sphericalToCartesian
import org.apache.spark.rdd.RDD
import com.astrolabsoftware.spark3d.geometryObjects._


/** Define palette of colors */
def colors : Array[java.awt.Color] = {
    Array(
        Color.BLACK, Color.RED, Color.GREEN, Color.BLUE,
        Color.ORANGE, Color.YELLOW, Color.DARK_GRAY, Color.PINK,
        Color.MAGENTA, Color.CYAN)
}

/** Define markers */
def markers : Array[Char] = {
    val strings = Array(
        ".", "s", "x", "+", "@", "q", 
        "-", "|", "O", "S", "#", "Q", "."
    )
    strings.map(x => x.toCharArray).flatten
    
}

/** 
  * format the data for smile.
  * The data for ScatterPlot must be Array[Array[Double]] (=Array[point3d])
  * We add one more dimension which is the partition.
  *
  * @param rdd : (RDD[Point3D])
  *   RDD whose elements are Point3D instances.
  * @return (Array[Array[Array[Double]]]) data as partitions -> points -> point -> coordinate 
  * 
  */
def format_data_for_smile(rdd: RDD[Point3D]) : Array[Array[Array[Double]]] = {
    rdd.map(
        x=> sphericalToCartesian(x).center.getCoordinate.toArray)
    .glom.collect().toArray
}

/** 
  * Plot 3D data sets.
  * 
  * @param display : (String)
  *   Either show or save. If save, extension will be given in the outname.
  * @param rddArr : (Array(RDD[Point3D]))
  *   Array containing RDD for data sets X whose elements are instances of Point3D.
  * part : (Int)
  *   Partition index to plot.
  * @param outname : (String)
  *   If save mode, name (incl. extenstion) for the out file.
  * @param title : (String)
  *   Title of the window.
  *
  */
def MyScatterPlotCross(
    display: String, rddArr: Array[RDD[Point3D]], 
    part: Int, outname: String, title: String) : Unit = {
    
    // Re-arange the data for plotting
    val dataInit = format_data_for_smile(rddArr(0))
    
    // Plot the results
    val window = ScatterPlot.plot(dataInit(part), markers(0), colors(0))
    
    for (pos <- 1 to rddArr.size - 1) {
      val dataOther = format_data_for_smile(rddArr(pos))
      window.points(dataOther(part), markers(pos), colors(pos))   
    }
    
    display match {
      case "show" => {
        val partFrame = new JFrame(title)
        partFrame.setLocationRelativeTo(null)
        partFrame.getContentPane().add(window)
        partFrame.setVisible(true)
        partFrame.setSize(new Dimension(500, 500))
      }
      case "save" => {
        val partHeadless = new Headless(window);
        partHeadless.pack();
        partHeadless.setVisible(true);
        partHeadless.setSize(new Dimension(500, 500))
        window.save(new java.io.File(outname))
      }
      case _ => throw new AssertionError("""
        I do not understand the kind of display you want.
        Choose between "show" and "save".
        """)
    }
}

// Set to "show" or "save"
val display = "show"
val partition = 0

// Display the result for healpix
MyScatterPlotCross(display, Array(pointRDDA.rawRDD,
                    sphereMatches.asInstanceOf[RDD[Point3D]]), partition, 
                    "querySphere.png", "Healpix Cross match (A, B, AxB)")
MyScatterPlotCross(display, Array(pointRDDA.rawRDD,
                    boxMatches.asInstanceOf[RDD[Point3D]]), partition, 
                    "queryBox.png", "Healpix Cross match (A, B, AxB)")